In [6]:
import tensorflow as tf
import keras
import cv2 
import os
import matplotlib.pyplot as plt
import numpy as np
import librosa

In [7]:
import pickle
import IPython.display as ipd
import sounddevice as sd
from scipy.io.wavfile import write
encoder = pickle.load(open("SerEncoder","rb"))
scaler = pickle.load(open('SerScaler', 'rb'))

In [8]:
model = keras.models.load_model('fser.h5')

In [9]:
def extract_features(data):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

In [20]:
freq = 44100
duration = 5
print('started')
recording = sd.rec(int(duration * freq), 
                   samplerate=freq, channels=2)
sd.wait()
print('ended')
write("SampleRecording3.wav", freq, recording)

started
ended


In [21]:
sample_rate= 44100
test1 = extract_features(recording.reshape(-1,))
test1 = np.array(test1)
test1 = np.expand_dims(test1, axis=0)
test1 = scaler.transform(test1)
test1 = np.expand_dims(test1, axis=2)
print(encoder.inverse_transform(model.predict(test1)))
ipd.Audio(recording.reshape(-1,),rate=freq*2)

[['happy']]
